In [1]:
import os
import shutil
import csv

In [2]:
#mi sposto nella cartella files
os.chdir('files')

In [3]:
#creo le sottocartelle...
for dirname in ['audio', 'docs', 'images']:
    if not os.path.exists(dirname):#...se queste non esistono
        os.makedirs(dirname)

#creo liste formati accettati
img_exts = ['.jpeg', '.jpg', '.png']
doc_exts = ['.txt', '.odt']
audio_exts = ['.mp3']

#se il file di recap...
if not os.path.exists('recap.csv'):#...non esiste
    recap = open('recap.csv', 'w', newline='')#lo creo: 'w' sta per 'write'
    writer = csv.writer(recap)
    writer.writerow(['name', 'type', 'size (B)'])#scrivo header del csv
else:#...esiste già
    recap = open('recap.csv', 'a', newline='')#lo apro in 'append' con 'a'
    writer = csv.writer(recap)
    
#itero dentro cartella corrente (cioè files: mi sono spostato dentro all'inizio)
for f in sorted(os.listdir('.')):
    
    #separo il nome del file dall'estensione
    name, ext = os.path.splitext(f)
    
    if ext in img_exts:#check estensione
        size = os.path.getsize(f)#calcolo dimensione in byte
        print(f'{name} type:image size:{size}B')#stampo info
        writer.writerow([name, 'image', size])#aggiorno recap
        shutil.move(f, 'images/')#sposto in sottocartella
        
    elif ext in doc_exts:
        size = os.path.getsize(f)
        print(f'{name} type:doc size:{size}B')
        writer.writerow([name, 'doc', size])
        shutil.move(f, 'docs/')
        
    elif ext in audio_exts:
        size = os.path.getsize(f)
        print(f'{name} type:audio size:{size}B')
        writer.writerow([name, 'audio', size])
        shutil.move(f, 'audio/')
        
    else:
        print('Formato non riconosciuto.')
    
recap.close()#è buona pratica chiudere un file dopo la scrittura

bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:doc size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B


In [4]:
import numpy as np
from PIL import Image
from tabulate import tabulate

In [5]:
table = []#creo tabella per tabulate

#ciclo dentro sottocartella images
for f in sorted(os.listdir('images')):
    #carico immagine
    img = Image.open(os.path.join('images', f))#uso join per costruire nome file relativo a sottocartella
    #trasformo immagine in array numpy
    img_array = np.array(img)
    
    #calcolo altezza e larghezza
    pixel_y, pixel_x = img_array.shape[:2]
    
    #caso scala di grigio
    if img_array.ndim == 2:  
        mean_gs = img_array.mean()#media dei valori dell'unico livello
        mean_r, mean_g, mean_b = 0, 0, 0#non esistono altri livelli
        mean_alpha = 0
        
    #caso RGB o RGBA: ho una dimensione aggiuntiva per i canali di colore (e quello di trasparenza)
    else:
        mean_gs = 0
        #calcolo della media per ogni canale:
        #- con la prima applicazione di mean faccio la media lungo l'asse y (righe)
        #- con la seconda, faccio la media lungo l'asse x (colonne)
        #il risultato è un array di 3 (RGB) o 4 (RGBA) numeri, corrispondenti alle
        #medie dei valori di ognuno dei 3 canali di colore e, se presente, di quello di trasparenza
        channels_mean = img_array.mean(axis=0).mean(axis=0)
        if len(channels_mean) == 3:#caso RGB, solo i tre canali di colore
            mean_alpha = 0    
            mean_r, mean_g, mean_b = channels_mean
        else:#caso RGBA, c'è anche il canale di trasparenza
            mean_r, mean_g, mean_b, mean_alpha = channels_mean
    
    #separo nome ed estensione
    name, ext = os.path.splitext(f)
    #costruisco riga della tabella tabulate
    row = [name, pixel_y, pixel_x, mean_gs, mean_r, mean_g, mean_b, mean_alpha]
    #aggiungo riga a tabella
    table += [row]

#stampo tabella tabulate
print(tabulate(table,
               headers=['name', 'height', 'width', 'grayscale', 'R', 'G', 'B', 'ALPHA'],#header
               tablefmt='fancy_grid',#formato
               floatfmt='.2f'))#formato numeri decimali

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
